In [11]:
import os
import requests
import pandas as pd
from datetime import datetime

# ---------------- CONFIG ----------------
API_URL = "https://api.eia.gov/v2/petroleum/pri/gnd/data/?frequency=monthly&data[0]=value&start=2023-01&end=2024-01&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key=HHp4XRvlkkuLIVV7xBwHQ0F7lDkO7cfN38mM4onx"  # <-- ใส่ API จริงของคุณ
RAW_FILE = "data/raw/petroleum_data.csv"
# ----------------------------------------

def fetch_data():
    """ดึงข้อมูลจาก API"""
    resp = requests.get(API_URL)
    resp.raise_for_status()
    data = resp.json()
    return data

def normalize_data(data):
    """แปลง JSON → DataFrame"""
    response = data.get("response", {})
    records = response.get("data", [])
    df = pd.DataFrame(records)

    # เพิ่ม timestamp ตอนดึงข้อมูล
    df["ingested_at"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return df

def save_to_csv(df_new):
    """บันทึกข้อมูลลง CSV (append ถ้ามีไฟล์เก่า)"""
    os.makedirs(os.path.dirname(RAW_FILE), exist_ok=True)

    if os.path.exists(RAW_FILE):
        df_old = pd.read_csv(RAW_FILE)

        # รวมข้อมูลทั้งหมด แล้วลบแถวซ้ำกัน (ป้องกัน duplicate period/series)
        df_all = pd.concat([df_old, df_new], ignore_index=True)
        df_all.drop_duplicates(subset=["period", "series"], keep="last", inplace=True)
    else:
        df_all = df_new

    df_all.to_csv(RAW_FILE, index=False, encoding="utf-8-sig")
    print(f"✅ Updated data saved to {RAW_FILE}")
    print(f"📊 Total rows: {len(df_all)}")

def main():
    print("🚀 Fetching data from API...")
    data = fetch_data()
    df_new = normalize_data(data)
    save_to_csv(df_new)

if __name__ == "__main__":
    main()


🚀 Fetching data from API...
✅ Updated data saved to data/raw/petroleum_data.csv
📊 Total rows: 9834
